# Calculate a climate index in a server hosting all the climate model data 

We will show here how to count the annual summer days for a particular geolocation of your choice using the results of a climate model, in particular, we can chose one of the historical or one of the shared socioeconomic pathway (ssp) experiments of the Coupled Model Intercomparison Project [CMIP6](https://pcmdi.llnl.gov/CMIP6/).

Thanks to the data and computer scientists Marco Kulüke, Fabian Wachsmann, Regina Kwee-Hinzmann, Caroline Arnold, Felix Stiehler, Maria Moreno, and Stephan Kindermann at DKRZ for their contribution to this notebook.

In this use case you will learn the following:
- How to access a dataset from the DKRZ CMIP6 model data archive
- How to count the annual number of summer days for a particular geolocation using this model dataset
- How to visualize the results


You will use:
- [Intake](https://github.com/intake/intake) for finding the data in the catalog of the DKRZ archive
- [Xarray](http://xarray.pydata.org/en/stable/) for loading and processing the data
- [hvPlot](https://hvplot.holoviz.org/index.html) for visualizing the data in the Jupyter notebook and save the plots in your local computer

## 0. Load Packages

In [ ]:
import numpy as np                    # fundamental package for scientific computing
import pandas as pd                   # data analysis and manipulation tool
import xarray as xr                   # handling labelled multi-dimensional arrays
import intake                         # to find data in a catalog, this notebook explains how it works
from ipywidgets import widgets        # to use widgets in the Jupyer Notebook
from geopy.geocoders import Nominatim # Python client for several popular geocoding web services
import folium                         # visualization tool for maps
import hvplot.pandas                  # visualization tool for interactive plots

## 1. Which dataset do we need? -> Choose Shared Socioeconomic Pathway, Place, and Year

<a id='selection'></a>

In [ ]:
# Produce the widget where we can select what experiment we are interested on 

experiments = {'historical':range(1850, 2015), 'ssp126':range(2015, 2101), 
               'ssp245':range(2015, 2101), 'ssp370':range(2015, 2101), 'ssp585':range(2015, 2101)}
experiment_box = widgets.Dropdown(options=experiments, description="Select  experiment: ", disabled=False,)
display(experiment_box)

In [ ]:
# Produce the widget where we can select what geolocation and year are interested on 

print("Feel free to change the default values.")

place_box = widgets.Text(description="Enter place:", value="Hamburg")
display(place_box)

x = experiment_box.value
year_box = widgets.Dropdown(options=x, description="Select year: ", disabled=False,)
display(year_box)

### 1.1 Find Coordinates of chosen Place
If ambiguous, the most likely coordinates will be chosen, e.g. "Hamburg" results in "Hamburg, 20095, Deutschland", (53.55 North, 10.00 East)

In [ ]:
# We use the module Nominatim gives us the geographical coordinates of the place we selected above

geolocator = Nominatim(user_agent="any_agent")
location = geolocator.geocode(place_box.value)

print(location.address)
print((location.latitude, location.longitude))

### 1.2 Show Place on a Map

In [ ]:
# We use the folium package to plot our selected geolocation in a map

m = folium.Map(location=[location.latitude, location.longitude])
tooltip = location.latitude, location.longitude
folium.Marker([location.latitude, location.longitude], tooltip=tooltip).add_to(m)
display(m)

We have defined the place and time. Now, we can search for the climate model dataset.

## 2. Intake Catalog

### 2.1 Load the Intake Catalog

In [ ]:
# Path to catalog descriptor on the DKRZ server
col_url = "/pool/data/Catalogs/mistral-cmip6.json"

# Open the catalog with the intake package and name it "col" as short for "collection"
col = intake.open_esm_datastore(col_url)

### 2.2 Browse the Intake Catalog
In this example we chose the Max-Planck Earth System Model in High Resolution Mode ("MPI-ESM1-2-HR") and the maximum temperature near surface ("tasmax") as variable. We also choose an experiment. CMIP6 comprises several kind of experiments. Each experiment has various simulation members. you can find more information in the [CMIP6 Model and Experiment Documentation](https://pcmdi.llnl.gov/CMIP6/Guide/dataUsers.html#5-model-and-experiment-documentation).

In [ ]:
# Store the name of the model we chose in a variable named "climate_model"

climate_model = "MPI-ESM1-2-HR" # here we choose Max-Plack Institute's Earth Sytem Model in high resolution

# This is how we tell intake what data we want

query = dict(
    source_id      = climate_model, # the model 
    variable_id    = "tasmax", # temperature at surface, maximum
    table_id       = "day", # daily maximum
    experiment_id  = experiment_box.label, # what we selected in the drop down menu,e.g. SSP2.4-5 2015-2100
    member_id      = "r10i1p1f1", # "r" realization, "i" initialization, "p" physics, "f" forcing
)

# Intake looks for the query we just defined in the catalog of the CMIP6 data pool at DKRZ
cat = col.search(**query)

del col

# Show query results
cat.df

### 2.3 Create Dictionary and get Data into one Object

In [ ]:
# cdf_kwargs are given to xarray.open_dataset
# cftime is like datetime but extends to all four digit years and many calendar types

dset_dict = cat.to_dataset_dict(cdf_kwargs={"chunks": {"time": -1}, "use_cftime": True})

In [ ]:
# get all data into one object

for key, value in dset_dict.items():
    model = key.split(".")[2]  # extract model name from key
    tasmax_xr = value["tasmax"].squeeze()  # extract variable from dataset

In [ ]:
tasmax_xr

## 3. Select Year and Look at (Meta) Data

In [ ]:
tasmax_year_xr = tasmax_xr.sel(time=str(year_box.value))

# Let's have a look at the xarray data array

tasmax_year_xr

We see not only the numbers, but also information about it, such as long name, units, and the data history. This information is called metadata.

## 4. Compare Model Grid Cell with chosen Location

In [ ]:
# Find nearest model coordinate by finding the index of the nearest grid point

abslat = np.abs(tasmax_year_xr["lat"] - location.latitude)
abslon = np.abs(tasmax_year_xr["lon"] - location.longitude)
c = np.maximum(abslon, abslat)

([xloc], [yloc]) = np.where(c == np.min(c)) # xloc and yloc are the indices of the neares model grid point

In [ ]:
# Draw map again

m = folium.Map(location=[location.latitude, location.longitude], zoom_start=8)


tooltip = location.latitude, location.longitude
folium.Marker(
    [location.latitude, location.longitude],
    tooltip=tooltip,
    popup="Location selected by You",
).add_to(m)

#
tooltip = float(tasmax_year_xr["lat"][yloc].values), float(tasmax_year_xr["lon"][xloc].values)
folium.Marker(
    [tasmax_year_xr["lat"][yloc], tasmax_year_xr["lon"][xloc]],
    tooltip=tooltip,
    popup="Model Grid Cell Center",
).add_to(m)

# Define coordinates of model grid cell (just for visualization)
rect_lat1_model = (tasmax_year_xr["lat"][yloc - 1] + tasmax_year_xr["lat"][yloc]) / 2
rect_lon1_model = (tasmax_year_xr["lon"][xloc - 1] + tasmax_year_xr["lon"][xloc]) / 2
rect_lat2_model = (tasmax_year_xr["lat"][yloc + 1] + tasmax_year_xr["lat"][yloc]) / 2
rect_lon2_model = (tasmax_year_xr["lon"][xloc + 1] + tasmax_year_xr["lon"][xloc]) / 2

# Draw model grid cell
folium.Rectangle(
    bounds=[[rect_lat1_model, rect_lon1_model], [rect_lat2_model, rect_lon2_model]],
    color="#ff7800",
    fill=True,
    fill_color="#ffff00",
    fill_opacity=0.2,
).add_to(m)

m

Climate models have a finite resolution. Hence, models do not provide the data of a particular point, but the mean over a model grid cell. Take this in mind when comparing model data with observed data (e.g. weather stations).


Now, we will visualize the daily maximum temperature time series of the model grid cell.

## 5. Draw Temperature Time Series and Count Summer days

The definition of a summer day varies from region to region. According to the [German Weather Service](https://www.dwd.de/EN/ourservices/germanclimateatlas/explanations/elements/_functions/faqkarussel/sommertage.html), "a summer day is a day on which the maximum air temperature is at least 25.0 °C". Depending on the place you selected, you might want to apply a different threshold to calculate the summer days index. 

In [ ]:
tasmax_year_place_xr = tasmax_year_xr[:, yloc, xloc] - 273.15 # Convert Kelvin to °C
tasmax_year_place_df = pd.DataFrame(index = tasmax_year_place_xr['time'].values, 
                                    columns = ['Temperature', 'Summer Day Threshold']) # create the dataframe

tasmax_year_place_df.loc[:, 'Model Temperature'] = tasmax_year_place_xr.values # insert model data into the dataframe
tasmax_year_place_df.loc[:, 'Summer Day Threshold'] = 25 # insert the threshold into the dataframe

# Plot data and define title and legend
tasmax_year_place_df.hvplot.line(y=['Model Temperature', 'Summer Day Threshold'], 
                                 value_label='Temperature in °C', legend='bottom', 
                                 title='Daily maximum Temperature near Surface for '+place_box.value, 
                                 height=500, width=620)

As we can see, the maximum daily temperature is highly variable over the year. As we are using the mean temperature in a model grid cell, the amount of summer days might we different that what you would expect at a single location.

In [ ]:
# Summer days index calculation
no_summer_days_model = tasmax_year_place_xr[tasmax_year_place_xr > 25].size # count the number of summer days

# Print results in a sentence
print("According to the German Weather Service definition, in the " +experiment_box.label +" experiment the " 
      +climate_model +" model shows " +str(no_summer_days_model) +" summer days for " +str(place_box.value) 
      + " in " + str(year_box.value) +".")

[Try another location and year](#selection)